In [1]:
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [ ]:
x = range(10)
''.join([f'{symbol}' for _ in x for symbol in '01'])

In [ ]:
years = range(2004, 2024)
urls = ["https://www.bgc-jena.mpg.de/wetter/mpi_roof_2003b.zip"]
urls += ([f'https://www.bgc-jena.mpg.de/wetter/mpi_roof_{year}{half}.zip' for year in years for half in 'ab'])
urls.append('https://www.bgc-jena.mpg.de/wetter/mpi_roof.zip')

In [5]:
urls

['https://www.bgc-jena.mpg.de/wetter/mpi_roof_2003b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2004a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2004b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2005a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2005b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2006a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2006b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2007a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2007b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2008a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2008b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2009a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2009b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2010a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2010b.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2011a.zip',
 'https://www.bgc-jena.mpg.de/wetter/mpi_roof_2011b.zip',
 'https://www.

In [7]:
weather = pd.DataFrame()

for url in urls:
    resp = urlopen(url)

    with ZipFile(BytesIO(resp.read())) as zfile:
        filename = zfile.namelist()[0]
        df = pd.read_csv(zfile.open(filename), sep=',', encoding='cp1252')

    weather = pd.concat([weather, df], ignore_index=True)

In [11]:
weather = tweak_weather(weather)

In [12]:
weather.to_csv('weather.csv', index=False)

In [ ]:
weather.to_parquet('weather.pqt')

In [6]:
def tweak_weather(weather):
    return (weather
        .rename(columns={'Date Time': 'date_time', 
                        'max. wv (m/s)': 'wv_max',
                        'max. PAR (µmol/m²/s)': 'PARmax'})
        .rename(columns=lambda c: c.split(' ')[0])
        .assign(date_time=lambda df: pd.to_datetime(df.date_time, format='%d.%m.%Y %H:%M:%S', utc='utc+1'))
        #.set_index('date_time')
    )